In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import glob

from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import (Input, Dense)

from sklearn import preprocessing

Using TensorFlow backend.


## Load data to tuple list

In [2]:
audio_len = 1500

if os.name == "nt":
    os.chdir("C:\\Users\\tymons\\001.Projects\\003.eul\\workspace\\smartula-analysis\\core\\csv")
else:
    os.chdir("/home/tymons/Projects/003.eul/workspace/smartula-analysis/csv")
    

all_filenames = [i for i in glob.glob("*.{}".format("csv"))]
#all_filenames = all_filenames[:600]
list_of_audios = []
for filename in all_filenames:
    samples = np.ravel(pd.read_csv(filename, header=None))
    array = np.array(samples[:audio_len]).astype(float)
    samples = array - array.mean()
    audio_tuple = (filename, samples)
    if len(samples) == audio_len:
        list_of_audios.append(audio_tuple)
    else:
        print("Error with: " + filename +". Got length:" + str(len(samples)))


Error with: 2019-06-06T03-23-11.csv. Got length:1000
Error with: 2019-06-07T03-09-11.csv. Got length:1000


## Model

In [8]:
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

sound_input = Input(shape=(1500,))
encoded = Dense(784, activation='relu')(sound_input)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)
encoder = encoded

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='relu')(decoded)
decoded = Dense(1500, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(sound_input, decoded)

In [10]:
autoencoder.compile(optimizer='adadelta', loss='mse')

## Normalization and Standarization


In [11]:
k = 0.8

data = np.array([audio[1] for audio in list_of_audios])
data = preprocessing.scale(data)
data = preprocessing.normalize(data)

index = int(k*len(data))
x_train = data[:index]
x_test = data[index:]



In [12]:
es = [EarlyStopping(monitor='val_loss', min_delta=1e-05, patience=2)]

autoencoder.fit(x_train, x_train,
                epochs=500,
                shuffle=True,
                batch_size = 100,
                validation_data=(x_test, x_test),
                callbacks = es)


Train on 855 samples, validate on 214 samples
Epoch 1/500
855/855 [==============================] - 2s 2ms/step - loss: 0.2498 - val_loss: 0.2500
Epoch 2/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2493 - val_loss: 0.2495
Epoch 3/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2488 - val_loss: 0.2490
Epoch 4/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2483 - val_loss: 0.2485
Epoch 5/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2478 - val_loss: 0.2480
Epoch 6/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2473 - val_loss: 0.2475
Epoch 7/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2468 - val_loss: 0.2470
Epoch 8/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2463 - val_loss: 0.2464
Epoch 9/500
855/855 [==============================] - 1s 1ms/step - loss: 0.2457 - val_loss: 0.2459
Epoch 10/500
855/855 [=======================

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encoded = encoder.predict(x_test)




In [ ]:
%matplotlib inline

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()